In [1]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI  # 나중에 모델을 바꿀때 이부분만 변경하면 ok

# default text-davinci-003
# llm = OpenAI()
# default gpt-3.5-turbo
chat = ChatOpenAI(
    temperature=0.1  # 0.1 -> 덜 창의적 0.9 -> 창의적 올라가고 무작위성도 올라감
)

# 이렇게 함수 통합 되어있는거 볼 수 있음
# llm.predict("물어볼꺼")
# chat.predict("물어볼꺼")

In [2]:
# message constuructor
# HumanMessage -> 사람이 보내는 메세지
# AIMessage -> AI 메세지
# SystemMessage -> AI의 기본 설정, 기본 값, 기본 context 지정 하는데 사용
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Korean",
    ),
    AIMessage(content="Ciao, mi chimo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?",
    ),
]

# chat.predict_messages(messages)

In [3]:
# ChatPromptTemplate -> template을 message로 부터 만듬
# PromptTemplate -> 그냥 String 을 통해서 만듬
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# {} -> 일종의 placeholder
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}.",
)

chat = ChatOpenAI(temperature=0.1)

prompt = template.format(country_a="Korea", country_b="Japan")

# chat.predict(prompt)

In [18]:

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Korea",
    name="Soctates",
    country_a="Korea",
    country_b="Japan",
)

chat.predict_messages(prompt)

AIMessage(content='안녕하세요! 제 이름은 AI입니다. 한국과 일본 사이의 거리는 대략 1,000km 정도입니다.')

In [4]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")  # 공백 제거 + string split array
        return list(map(str.strip, items))


p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [5]:
template = ChatPromptTemplate.from_messages([
        ("system","You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do Not reply with anything else."),
        ("human", "{question}"),
])

prompt = template.format_messages(max_items=10 , question = "Battle.net 에는 어떤 게임이 있어?")

result = chat.predict_messages(prompt)
print(result)

p = CommaOutputParser()

p.parse(result.content)

content='overwatch, world of warcraft, hearthstone, diablo, starcraft, heroes of the storm, warcraft iii, call of duty, destiny, diablo ii'


['overwatch',
 'world of warcraft',
 'hearthstone',
 'diablo',
 'starcraft',
 'heroes of the storm',
 'warcraft iii',
 'call of duty',
 'destiny',
 'diablo ii']

In [9]:
# chain
# 모든 요소를 하나로 합치는 것
# 합쳐진 요소들은 하나의 chain으로 실행됨 하나하나 순서대로 result를 반환할때 까지

# | <- langchain
# Template , Language Model , Output Parser 으로 구성된 chain
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 2,
    "question": "What are the hoshimachi suisei songs?"
})

['stella', 'next color planet', 'Ochame Kinou']